<a href="https://colab.research.google.com/github/donghwi2022/ds-sa-cp2-phase2/blob/%EC%8B%A0%EB%8F%99%ED%9C%98/model_annoy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 import 및 사용 데이터 load

In [1]:
!pip install --user annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import annoy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_parquet("/content/drive/MyDrive/cp2_phase2_data/fashion_logs.parquet")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201009 entries, 0 to 1254584
Data columns (total 23 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   created_at          1201009 non-null  category
 1   session_id          1201009 non-null  int32   
 2   payment_method      1201009 non-null  category
 3   promo_amount        1201009 non-null  int32   
 4   promo_code          1201009 non-null  category
 5   shipment_fee        1201009 non-null  int32   
 6   total_amount        1201009 non-null  int32   
 7   product_id          1201009 non-null  int32   
 8   productDisplayName  1201009 non-null  category
 9   quantity            1201009 non-null  int32   
 10  item_price          1201009 non-null  int32   
 11  product_gender      1201009 non-null  category
 12  baseColor           1201009 non-null  category
 13  season              1201009 non-null  category
 14  year                1201009 non-null  int32   
 15

In [5]:
data.head()

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
0,2018-07-29T15:22:01,194913,Debit Card,1415,WEEKENDSERU,10000,199832,54728,Vans Men Black Shoes,1,...,Summer,2012,Casual,Footwear Shoes Casual Shoes,5868,F,2000-08-20,Android,Jakarta Raya,2018-07-28
1,2018-07-30T12:40:22,155874,Credit Card,0,No,10000,155526,16193,Puma Men Knitted Vest Green Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
2,2018-09-15T11:51:17,489413,OVO,0,No,10000,550696,53686,Kiara Women Purple & Yellow Handbag,4,...,Summer,2012,Casual,Accessories Bags Handbags,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
3,2018-11-01T11:23:48,627839,Credit Card,0,No,0,271012,20228,Wrangler Women Cable Red Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
4,2018-12-18T11:20:30,742846,Credit Card,0,No,0,198753,55220,Lakme Absolute Forever Silk Chestnut Lip Liner 03,1,...,Spring,2017,Casual,Personal Care Lips Lip Liner,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28


# 2. CB(annoy) 모델 구축

## 1. 학습 데이터와 테스트 데이터 분리



In [145]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=2)

In [146]:
train_data

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
880339,2016-10-29T16:31:09,348137,LinkAja,3840,WEEKENDSERU,10000,456089,42098,ADIDAS Men Grey & Blue Chelsea Polo T-shirt,1,...,Summer,2012,Casual,Apparel Topwear Tshirts,55524,F,2002-08-21,iOS,Lampung,2016-07-09
780643,2017-08-27T23:22:43,607642,Credit Card,8468,WEEKENDMANTAP,0,71551,9149,Lee Women Rinse Blue Jeans,1,...,Summer,2013,Casual,Apparel Bottomwear Jeans,44360,M,1991-09-17,iOS,Jawa Timur,2016-07-23
62986,2022-04-13T04:08:34,401515,OVO,1794,AZ2022,0,189592,42625,Estelle Women Earrings,1,...,Winter,2016,Casual,Accessories Jewellery Earrings,68215,F,2004-12-20,Android,Kalimantan Timur,2020-07-05
1036465,2021-03-30T01:05:59,257619,Credit Card,0,No,10000,1157830,44138,Dunhill Men Desire Blue Perfume,1,...,Spring,2017,Casual,Personal Care Fragrance Perfume and Body Mist,16069,F,2006-05-10,Android,Jawa Tengah,2017-06-30
662132,2022-05-24T05:31:32,28323,OVO,8314,AZ2022,10000,1292730,40650,Span Women Maroon & Black Kurta,1,...,Summer,2012,Ethnic,Apparel Topwear Kurtas,80697,M,1999-01-20,iOS,Nusa Tenggara Barat,2017-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151638,2019-06-07T08:59:56,104185,OVO,6620,AZ2022,10000,2236402,14120,Flying Machine Women Solid Grey Tops,1,...,Fall,2011,Casual,Apparel Topwear Tops,76387,F,1999-09-30,iOS,Jakarta Raya,2018-10-16
1005168,2022-03-27T22:42:54,417357,Credit Card,3054,LIBURDONG,0,553998,33280,Gini and Jony Kids Unisex Blue Jeans,2,...,Summer,2012,Casual,Apparel Bottomwear Jeans,69096,F,1991-03-02,Android,Sulawesi Utara,2021-05-30
1183434,2018-05-24T16:44:33,504605,OVO,0,No,50000,500396,14614,United Colors of Benetton Men Pack of 2 Navy B...,1,...,Summer,2011,Casual,Accessories Socks Socks,11445,M,1992-11-08,Android,Kalimantan Tengah,2016-07-13
212104,2021-08-25T15:22:28,619970,Debit Card,1477,AZ2022,10000,164079,33937,ADIDAS Men Polo Black T-shirt,1,...,Summer,2012,Casual,Apparel Topwear Tshirts,20067,M,2001-05-02,Android,Jawa Tengah,2018-07-29


In [147]:
test_data

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
844403,2022-06-12T01:54:39,825616,Credit Card,0,No,10000,1970650,54410,Baggit Women Pink Wallet,1,...,Summer,2012,Casual,Accessories Wallets Wallets,61017,M,2000-09-16,iOS,Kepulauan Riau,2017-08-02
11563,2022-05-20T02:00:07,760117,Debit Card,0,No,10000,169354,21677,Fossil Men Black Dial Chronograph Watch CH2674,1,...,Winter,2016,Casual,Accessories Watches Watches,40893,F,2003-04-29,Android,Lampung,2017-07-16
963571,2021-07-07T01:08:44,176866,Gopay,4301,XX2022,10000,274648,7163,Scullers Men's Scul Navy T-shirt,1,...,Fall,2011,Casual,Apparel Topwear Tshirts,59297,F,1980-01-28,Android,Jawa Barat,2017-11-30
110413,2020-10-06T01:53:56,545697,Credit Card,0,No,0,72144,6383,Numero Uno Men's Navy Blue Casual Shoe,1,...,Summer,2011,Casual,Footwear Shoes Casual Shoes,23523,F,2005-05-26,Android,Jakarta Raya,2018-07-08
37903,2018-11-03T02:46:38,284575,Debit Card,1503,WEEKENDSERU,15000,587889,28267,Gatsby Men Original Deo,7,...,Spring,2017,Casual,Personal Care Fragrance Deodorant,87804,F,2010-11-24,Android,Jawa Timur,2018-09-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438738,2017-10-24T18:21:52,36220,Gopay,6601,BUYMORE,10000,359019,6789,Timberland Men Meriden White Check Shirt,1,...,Summer,2011,Casual,Apparel Topwear Shirts,14956,M,1990-08-17,Android,Jawa Barat,2017-10-08
976393,2022-07-27T22:53:12,386539,Gopay,0,No,0,2791490,59088,Peri Peri Women Purple Bra,1,...,Summer,2017,Casual,Apparel Innerwear Bra,8272,F,1995-07-23,Android,Jakarta Raya,2022-06-12
1250579,2022-07-02T18:39:08,835206,Gopay,0,No,10000,415752,58795,Highlander Men Red Striped Shirt,2,...,Fall,2012,Smart Casual,Apparel Topwear Shirts,54590,M,2002-01-12,Android,Kalimantan Barat,2022-05-02
461887,2021-07-24T08:55:06,783125,OVO,0,No,10000,3954937,56607,Streetwear Shine Divine Whitening Natural Compact,1,...,Spring,2017,Casual,Personal Care Makeup Compact,32352,M,1996-08-25,Android,Jakarta Raya,2019-09-15


## 2. 벡터 유사도 계산
- OOM 문제를 해결하기 위해 코사인 유사도가 대신 벡터 유사도인 annoy모델 사용

In [129]:
# 사용할 feature 선별
product_data = data[['product_id', 'product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].drop_duplicates('product_id')
product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']] = product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].astype('str')
product_data.reset_index(drop=True, inplace=True)
product_data

,product_id,product_gender,baseColor,season,year,usage,Category
0,54728,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes
1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters
2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags
3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters
4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner
...,...,...,...,...,...,...,...
44440,42430,Women,Blue,Summer,2012,Ethnic,Apparel Topwear Kurtis
44441,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas
44442,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes
44443,36651,Women,Red,Summer,2012,Casual,Accessories Wallets Wallets


In [130]:
product_data['features'] = product_data['product_gender']+' ' +product_data['baseColor']+' ' +product_data['season']+' ' +product_data['year']+' ' +product_data['usage']+' ' +product_data['Category']
product_data

,product_id,product_gender,baseColor,season,year,usage,Category,features
0,54728,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters,Men Green Fall 2011 Casual Apparel Topwear Swe...
2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags,Women Purple Summer 2012 Casual Accessories Ba...
3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters,Women Red Fall 2011 Casual Apparel Topwear Swe...
4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner,Women Brown Spring 2017 Casual Personal Care L...
...,...,...,...,...,...,...,...,...
44440,42430,Women,Blue,Summer,2012,Ethnic,Apparel Topwear Kurtis,Women Blue Summer 2012 Ethnic Apparel Topwear ...
44441,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas,Women Brown Summer 2012 Ethnic Apparel Topwear...
44442,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes,Men Brown Fall 2012 Casual Footwear Shoes Casu...
44443,36651,Women,Red,Summer,2012,Casual,Accessories Wallets Wallets,Women Red Summer 2012 Casual Accessories Walle...


In [132]:
# tf-idf 벡터화
tfidf = TfidfVectorizer()
product_tfidf = tfidf.fit_transform(product_data['features'])
product_tfidf = pd.DataFrame(product_tfidf.todense(), columns = tfidf.get_feature_names_out())
product_tfidf

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,...,waistcoat,wallets,wash,watches,water,white,winter,women,wristbands,yellow
0,0.0,0.0,0.0,0.0,0.000000,0.257638,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.251266,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.243786,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.227443,0.0,0.0
3,0.0,0.0,0.0,0.0,0.252014,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.216338,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.130091,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44440,0.0,0.0,0.0,0.0,0.000000,0.226641,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.211448,0.0,0.0
44441,0.0,0.0,0.0,0.0,0.000000,0.256879,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.239658,0.0,0.0
44442,0.0,0.0,0.0,0.0,0.000000,0.240922,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
44443,0.0,0.0,0.0,0.0,0.000000,0.177587,0.0,0.0,0.0,0.0,...,0.0,0.861378,0.0,0.0,0.0,0.0,0.0,0.165682,0.0,0.0


In [133]:
# annoy 모델 초기화
annoy_index = annoy.AnnoyIndex(f=product_tfidf.shape[1], metric='angular')
product_data_arr = product_tfidf.to_numpy()

for idx in range(product_data_arr.shape[0]):
    vector = product_data_arr[idx, :]
    annoy_index.add_item(idx, vector)

In [134]:
# annoy 모델 구축(트리의 개수 = 5)
annoy_index.build(n_trees = 5) 
annoy_index.save('product.ann')

True

In [148]:
# 인덱스가 0인 상품(product_id=54728)에 대해 유사한 상품 20개 추천
product_vec = product_tfidf.iloc[0, :]
get_nns_list = annoy_index.get_nns_by_vector(vector=product_vec, n=21, include_distances=False) # 유사한 벡터를 찾기 때문에 본인 포함
product_data.iloc[get_nns_list[1:]]

,product_id,product_gender,baseColor,season,year,usage,Category,features
560,38548,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
2269,49060,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
2575,26618,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
2706,39640,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5178,43881,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5240,42802,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5474,29568,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
5567,38525,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
6078,40468,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
7288,35660,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...


In [174]:
# 고객 id를 입력받아 가장 최근 구매한 상품과 유사한 상품 20개를 추천하는 함수
def product_recomm (customer_id) :
    if customer_id not in train_data['customer_id'].unique() :
        pass
    else :
        latest_data = train_data[train_data['customer_id'] == customer_id].sort_values('created_at').tail(1)
        latest_product = latest_data['product_id'].values[0]
        product_idx = product_data[product_data['product_id'] == latest_product].index[0]

        product_vec = product_tfidf.iloc[product_idx, :]
        get_nns_list = annoy_index.get_nns_by_vector(vector=product_vec, n=21, include_distances=False)
        return product_data.iloc[get_nns_list[1:]]

In [175]:
# 고객 id가 89369인 고객으로 테스트
recomm_data = product_recomm(89369)
recomm_data

,product_id,product_gender,baseColor,season,year,usage,Category,features
3369,36248,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
4408,25549,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
4732,38792,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
4906,31257,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
5165,37097,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
5813,38623,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
6673,26494,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
7136,44867,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
8172,23647,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...
8364,46406,Men,Green,Summer,2012,Casual,Apparel Topwear Tshirts,Men Green Summer 2012 Casual Apparel Topwear T...


# 3. 성능 평가

In [176]:
# precision@k 계산 함수
def precision_k(test_data, recommend_data) : 
    recommend_list = recommend_data['product_id'].to_list()
    common_product = test_data[test_data['product_id'].isin(recommend_list)]

    return len(common_product) / len(recommend_list)

# recall@k 계산 함수 
def recall_k(test_data, recommend_data) : 
    recommend_list = recommend_data['product_id'].to_list()
    common_product = test_data[test_data['product_id'].isin(recommend_list)]

    return len(common_product) / len(test_data)

In [177]:
# 고객 id가 29496인 샘플 데이터로 테스트
sample_data = test_data[test_data['customer_id'] == 89369]

print(precision_k(sample_data, recomm_data))
print(recall_k(sample_data, recomm_data))

0.0
0.0


In [178]:
# 모든 고객 id에 대해서 테스트
customer_list = test_data['customer_id'].unique()
precision_total = 0
recall_total = 0

for customer_id in customer_list : 
    sample_data = test_data[test_data['customer_id'] == customer_id]
    recomm_data = product_recomm(customer_id)
    if recomm_data is None :
        pass
    else : 
        precision_total += precision_k(sample_data, recomm_data)
        recall_total += recall_k(sample_data, recomm_data)

print("precision :", precision_total/len(customer_list))
print("reall :", recall_total/len(customer_list))

precision : 0.0001613899180419891
reall : 0.0004927032935038315
